In [11]:
import numpy
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from fpdf import FPDF
import os
import pandas as pdf
from tkinter import *
from tkinter import filedialog

In [9]:
def getcpt(path):
    data=np.loadtxt(path)
    n=int(data.shape[0]/2)
    Data=np.zeros((n,2))
    for i in range(0,n):
        Data[i,0]=data[2*i]
        Data[i,1]=data[2*i+1]
    index=np.argwhere(Data==0)[:,0]
    array=np.unique(index)
    Data=np.delete(Data,array,axis=0)
    d=int(Data.shape[0])
    if d<300:
        print('size error')
        print('{}'.format(path))
        cpt=np.zeros((300,3))
        cpt[:d,0]=Data[:d,1]/Data[:d,0]/10
        cpt[:d,1]=Data[:d,0]*10
        cpt[d:299,:]=cpt[d-1,:]
    else:
        cpt=np.zeros((d,3))
        cpt[:,0]=Data[:,1]/Data[:,0]/10
        cpt[:,1]=Data[:,0]*10
    return cpt,d
def cycling(path):
    TI=np.arange(0.1,30.1,0.1)
    for cur_path, cur_dirs, cur_files in os.walk(path):
        for cur_dir in cur_dirs:
            for path, cur_dirs, cur_files in os.walk(os.path.join(cur_path,cur_dir)):
                for cur_file in cur_files:
                    path = os.path.join(cur_path, cur_dir)
                    path = os.path.join(path, cur_file)
                    # print(path)
                    cpt,d=getcpt(path)
                    cpt=classification(cpt,d)
                    # print(cpt[0:299,2])
                    TI=np.column_stack((TI,cpt[0:300,2]))
    print(TI.shape)
                    # np.savetxt(os.path.join(cur_path, cur_file),cpt)
    return  TI
def iszeros(TI):
    index=np.argwhere(TI==0)
    for i in index:
        if i[1]==0:
            TI[i[0],i[1]]= TI[i[0],i[1]+1]
        else:
            TI[i[0],i[1]]= TI[i[0],i[1]-1]
def filepath():
    root1 = Tk()
    root1.withdraw()
    Folderpath = filedialog.askdirectory()
    # filepath=filedialog.askopenfilename()
    return Folderpath

def queren():
    Folderpath = filepath()
    TI=cycling(Folderpath)
    iszeros(TI)
    np.savetxt('{}/TI.txt'.format(Folderpath),TI,'%.1f')
    TI=pd.DataFrame(TI)
    TI.to_excel('{}/TI.xlsx'.format(Folderpath))
def exit():
    root.quit()

root = Tk()
root.title('请输入文件目录')
root.geometry('320x320')
btn2 = Button(root, text='选择文件夹', command=queren)
btn2.place(relx=0.35, rely=0.87, in_=root)
# btn.pack()
btn3 = Button(root, text='退出', command=exit)
btn3.place(relx=0.65, rely=0.87, in_=root)
# btn.pack()
root.mainloop()

size error
D:/地勘数据CPT钻孔/6号线静探分工点\Q28\SJQ28XJ10.TXT
size error
D:/地勘数据CPT钻孔/6号线静探分工点\S27\SJS27XJ13.TXT
size error
D:/地勘数据CPT钻孔/6号线静探分工点\S27\SJS27XJ16.TXT
size error
D:/地勘数据CPT钻孔/6号线静探分工点\S29\SJS29XJ09.TXT
(300, 170)
size error
D:/地勘数据CPT钻孔/6号线\S2\SJS2XJ05.TXT
(300, 197)
(300, 191)


KeyboardInterrupt: 

In [3]:
def classification(cpt,d):
    for i in range(0,d):
        y=float(cpt[i,1])
        if cpt[i,0]> 10:
            cpt[i,0]=cpt[i-1,0]+0.1
        x=float(cpt[i,0])
        y1=-0.4996+11.7931*0.29094**x
        y2=0.83636*math.pow(1.25008,x)
        y3=math.exp(1.16653+0.57542*x-0.01407*x*x)
        y4=math.exp(1.93435+0.83996*x-0.04413*x*x)
        y5=math.exp(3.17279+1.04622*x-0.05286*x*x)
        y6=math.exp(4.86044+1.63056*x-0.32427*x*x)
        y7=math.exp(9.57894-2.35758*x+0.27909*x*x)
        y8=56.07598+704.2389*math.pow(0.48454,x)
        y9=math.exp(48.63502-22.08261*x+2.77615*x*x)
        if x>0.1 and x<1.67 and y-y1 <=0:
            cpt[i,2]=1
        if 1.66<x<10 and y-y2<=0 and y-y1>0:
            cpt[i,2]=2
        if 0.73<x<10.0 and y-y2>0 and y-y3<=0 and y-y1>0 and y-y8<=0:
            cpt[i,2]=3
        if 0.32<x<5.91 and y-y3>0 and y-y4<=0 and y-y1>0 and y-y8<=0:
            cpt[i,2]=4
        if 0.1<x<3.82 and y-y4>0 and y-y5<=0 and y-y1>0 and y-y7<=0:
            cpt[i,2]=5
        if 0.1<x<2.46 and y-y5>0 and y-y6<=0 and y<1000 and y-y7<=0:
            cpt[i,2]=6
        if 0.1<x<1.03 and y-y6>0:
            cpt[i,2]=7
        if 1.39<x<4.82 and y-y7>0 and y-y9>0:
            cpt[i,2]=8
        if 3.82<x<10 and y-y8>0 and y-y9<=0:
            cpt[i,2]=9
    return cpt

In [14]:
# for cur_path, cur_dirs, cur_files in os.walk('D:\地勘数据CPT钻孔/6C\（Q23）静探'):
#      for pdf_file in cur_files:
#         path = os.path.join(cur_path, pdf_file)
#         # print(path)
#         data=np.loadtxt(path)
#         # data=data.reshape(-1)
#         # np.savetxt(path,data,'%.4f')
# data=np.loadtxt('D:\地勘数据CPT钻孔/6C\（Q23）静探/Q23XJ04.txt')
# n=int(data.shape[0]/2)
# Data=np.zeros((n,2))
# for i in range(0,n):
#     Data[i,0]=data[2*i]
#     Data[i,1]=data[2*i+1]
# index=np.argwhere(Data==0)[:,0]
# array=np.unique(index)
# Data=np.delete(Data,array,axis=0)
# d=int(Data.shape[0])
# cpt=np.zeros((d,3))
# cpt[:,0]=Data[:,1]/Data[:,0]/10
# cpt[:,1]=Data[:,0]*10
# cpt=classification(cpt,d)
import xlrd
data=np.loadtxt('D:\地勘数据CPT钻孔/6C/TI.txt')
data=pd.DataFrame(data)
data.to_excel('D:\地勘数据CPT钻孔/6C/TI.xlsx')